In [341]:
import os
import numpy as np
import pandas as pd
from transformers import AutoTokenizer
from tqdm import tqdm
from typing import Iterable, Callable
os.environ["LOGGING_INI"] = "../logging.ini"
from questionanswering import squad

In [342]:
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [343]:
df = squad.parse_json_file("input/train-v2.0.json")
questions = list(df["question"])
contexts = list(df["context"])
df.info()

100%|█████████████████████████████████| 442/442 [00:08<00:00, 54.32it/s]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130319 entries, 0 to 130318
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            130319 non-null  object
 1   title         130319 non-null  object
 2   question      130319 non-null  object
 3   answer_text   130319 non-null  object
 4   answer_start  130319 non-null  int16 
 5   context       130319 non-null  object
dtypes: int16(1), object(5)
memory usage: 5.2+ MB


In [344]:
assert len(df) == 130319
assert len(df) == len(set(df["id"]))
df.set_index("id", drop=False, inplace=True)
df.head()

,id,title,question,answer_text,answer_start,context
id,,,,,,
56be85543aeaaa14008c9063,56be85543aeaaa14008c9063,Beyoncé,when did beyonce start becoming popular?,in the late 1990s,280,"beyoncé giselle knowles - carter / biːˈjɒnseɪ / bee - yon - say born september 4 , 1981 is an american singer , songwriter , record producer and actress . born and raised in houston , texas , she performed in various singing and dancing competitions as a child , and rose to fame in the late 1990s as lead singer of r&b girl - group destiny's child . managed by her father , mathew knowles , the group became one of the world's best - selling girl groups of all time . their hiatus saw the release of beyoncé's debut album , dangerously in love 2003 , which established her as a solo artist worldwide , earned five grammy awards and featured the billboard hot 100 number - one singles crazy in love and baby boy ."
56be85543aeaaa14008c9065,56be85543aeaaa14008c9065,Beyoncé,what areas did beyonce compete in when she was growing up?,singing and dancing,217,"beyoncé giselle knowles - carter / biːˈjɒnseɪ / bee - yon - say born september 4 , 1981 is an american singer , songwriter , record producer and actress . born and raised in houston , texas , she performed in various singing and dancing competitions as a child , and rose to fame in the late 1990s as lead singer of r&b girl - group destiny's child . managed by her father , mathew knowles , the group became one of the world's best - selling girl groups of all time . their hiatus saw the release of beyoncé's debut album , dangerously in love 2003 , which established her as a solo artist worldwide , earned five grammy awards and featured the billboard hot 100 number - one singles crazy in love and baby boy ."
56be85543aeaaa14008c9066,56be85543aeaaa14008c9066,Beyoncé,when did beyonce leave destiny's child and become a solo singer?,2003,545,"beyoncé giselle knowles - carter / biːˈjɒnseɪ / bee - yon - say born september 4 , 1981 is an american singer , songwriter , record producer and actress . born and raised in houston , texas , she performed in various singing and dancing competitions as a child , and rose to fame in the late 1990s as lead singer of r&b girl - group destiny's child . managed by her father , mathew knowles , the group became one of the world's best - selling girl groups of all time . their hiatus saw the release of beyoncé's debut album , dangerously in love 2003 , which established her as a solo artist worldwide , earned five grammy awards and featured the billboard hot 100 number - one singles crazy in love and baby boy ."
56bf6b0f3aeaaa14008c9601,56bf6b0f3aeaaa14008c9601,Beyoncé,in what city and state did beyonce grow up?,"houston , texas",174,"beyoncé giselle knowles - carter / biːˈjɒnseɪ / bee - yon - say born september 4 , 1981 is an american singer , songwriter , record producer and actress . born and raised in houston , texas , she performed in various singing and dancing competitions as a child , and rose to fame in the late 1990s as lead singer of r&b girl - group destiny's child . managed by her father , mathew knowles , the group became one of the world's best - selling girl groups of all time . their hiatus saw the release of beyoncé's debut album , dangerously in love 2003 , which established her as a solo artist worldwide , earned five grammy awards and featured the billboard hot 100 number - one singles crazy in love and baby boy ."
56bf6b0f3aeaaa14008c9602,56bf6b0f3aeaaa14008c9602,Beyoncé,in which decade did beyonce become famous?,late 1990s,287,"beyoncé giselle knowles - carter / biːˈjɒnseɪ / bee - yon - say born september 4 , 1981 is an american singer , songwriter , record producer and actress . born and raised in houston , texas , she performed in various singing and dancing competitions as a child , and rose to fame in the late 1990s as lead singer of r&b girl - group destiny's child . managed by her father , mathew knowles , the group became one of the world's best - selli

In [345]:
s = pd.Series(np.where(df["answer_start"]==-1, True, False))
assert s.value_counts()[True] == 43498
s.value_counts(normalize=True)

False    0.666219
True     0.333781
dtype: float64

# Answer corrections

In [346]:
%%time
df.loc["56bf7e603aeaaa14008c9681", "answer_text"] = "split with Luckett and Roberson"
df.loc["56be973d3aeaaa14008c9123", "answer_text"] = "sixth"
df.loc["56d3ac8e2ccc5a1400d82e1b", "answer_text"] = "operatic"
df.loc["56cbdea66d243a140015edae", "answer_text"] = "seven"
df.loc["56cbdea66d243a140015edae", "answer_start"] = 339
df.loc["56cf609aaab44d1400b89187", "answer_text"] = "seven"
df.loc["56cf609aaab44d1400b89187", "answer_start"] = 339
df.loc["56d20a6ae7d4791d0090261a", "answer_text"] = "seven"
df.loc["56d20a6ae7d4791d0090261a", "answer_start"] = 339
df.loc["56ccde7862d2951400fa64d9", "answer_start"] = 798
df.loc["56cd682162d2951400fa658e", "answer_start"] = 467
df.loc["56ce726faab44d1400b88793", "answer_text"] = "three"
df.loc["56ce726faab44d1400b88793", "answer_start"] = 308
df.loc["56cc57466d243a140015ef24", "question"] = "in which year did the sales of iPhone exceed iPod?"
df.loc["56cc57466d243a140015ef24", "answer_text"] = "2010"
df.loc["56cc57466d243a140015ef24", "answer_start"] = 449
df.loc["56ce750daab44d1400b887b4", "answer_start"] = 406
df.loc["56cd5d3a62d2951400fa653e", "answer_text"] = "manually"
df.loc["56cd8ffa62d2951400fa6723", "question"] = "what is Akiko Kōmoto's nationality?"
df.loc["56cd8ffa62d2951400fa6723", "answer_text"] = "Japanese"
df.loc["56cf2e15aab44d1400b88dc9", "answer_text"] = "twenty-fourth"
df.loc["56cf2e15aab44d1400b88dcb", "answer_text"] = "fourth"
df.loc["56cf2e15aab44d1400b88dcb", "answer_start"] = 114
df.loc["56cdcf7d62d2951400fa686d", "answer_start"] = 532
df.loc["56cdcf7d62d2951400fa686e", "answer_start"] = 581
df.loc["56cf4113aab44d1400b88ee0", "answer_text"] = "fourth"
df.loc["56cf469eaab44d1400b88f00", "answer_text"] = "sixth"
df.loc["56cde1f462d2951400fa6961", "answer_text"] = "two"
df.loc["56cde1f462d2951400fa6961", "answer_start"] = 551
df.loc["56cebbdeaab44d1400b8895c", "answer_text"] = "a million"
df.loc["56cebbdeaab44d1400b8895c", "answer_start"] = 275
df.loc["56cf1a05aab44d1400b88d7f", "answer_start"] = 342
df.loc["56cf3442aab44d1400b88e39", "answer_start"] = 379
df.loc["56cfe987234ae51400d9c09b", "answer_text"] = "92"
df.loc["56cfe987234ae51400d9c09b", "answer_start"] = 452
df.loc["56cf36e5aab44d1400b88e71", "answer_start"] = 652
df.loc["56cffb63234ae51400d9c1e7", "answer_text"] = "two"
df.loc["56cffb63234ae51400d9c1e7", "answer_start"] = 141
df.loc["56cf572daab44d1400b89072", "answer_text"] = "41st"
df.loc["56cf5c2caab44d1400b8911d", "answer_start"] = 170
df.loc["56d3f37f2ccc5a1400d82f85", "answer_start"] = 170
df.loc["56cf4e9aaab44d1400b88f9c", "answer_text"] = "seventh"
df.loc["56cf4e9aaab44d1400b88f9c", "answer_start"] = 1006
df.loc["56d0f47a17492d1400aab69d", "answer_text"] = "seventh"
df.loc["56d0f47a17492d1400aab69d", "answer_start"] = 1006
df.loc["56cf58f7aab44d1400b890b8", "answer_text"] = "three"
df.loc["56cf58f7aab44d1400b890b8", "answer_start"] = 288
df.loc["56d0f8a117492d1400aab6af", "answer_text"] = "three"
df.loc["56d0f8a117492d1400aab6af", "answer_start"] = 288
df.loc["56cf64d24df3c31400b0d6f5", "answer_text"] = "three"
df.loc["56cf64d24df3c31400b0d6f5", "answer_start"] = 940
df.loc["56d1042317492d1400aab72f", "answer_text"] = "three"
df.loc["56d1042317492d1400aab72f", "answer_start"] = 940
df.loc["56cf657b4df3c31400b0d6ff", "answer_text"] = "two"
df.loc["56cf657b4df3c31400b0d6ff", "answer_start"] = 99
df.loc["56d1056017492d1400aab755", "answer_start"] = 1090
df.loc["56cf79be4df3c31400b0d80e", "answer_text"] = "two"
df.loc["56cf79be4df3c31400b0d80e", "answer_start"] = 381
df.loc["56d11a9217492d1400aab976", "answer_text"] = "two"
df.loc["56d11a9217492d1400aab976", "answer_start"] = 381
df.loc["56cf829c234ae51400d9bdd4", "answer_text"] = "two"
df.loc["56cf829c234ae51400d9bdd4", "answer_start"] = 606
df.loc["56d131ad17492d1400aabbd3", "answer_text"] = "two"
df.loc["56d131ad17492d1400aabbd3", "answer_start"] = 964
df.loc["56cf8915234ae51400d9be15", "answer_text"] = "three"
df.loc["56cf8915234ae51400d9be15", "answer_start"] = 77
df.loc["56d13983e7d4791d00902041", "answer_text"] = "three"
df.loc["56d13983e7d4791d00902041", "answer_start"] = 77
df.loc["56d1b926e7d4791d009020cf", "answer_text"] = "four"
df.loc["56d1b926e7d4791d009020cf", "answer_start"] = 792
df.loc["56d1c2d2e7d4791d00902121", "answer_text"] = "5th century ce"
df.loc["56d24a6fb329da140004ed00", "answer_start"] = 521
df.loc["56d2641d59d6e41400145f54", "answer_start"] = 454
df.loc["56d366ee59d6e414001462f6", "answer_text"] = "fifteenth"
df.loc["56d366ee59d6e414001462f6", "answer_start"] = 342
df.loc["56d36e4b59d6e4140014638c", "answer_text"] = "fourteenth"
df.loc["56db2190e7c41114004b4dd5", "answer_text"] = "fourth"
df.loc["56d383b159d6e414001465e6", "answer_text"] = "fifteenth"
df.loc["56d383b159d6e414001465e6", "answer_start"] = 40
df.loc["56d383b159d6e414001465e7", "answer_text"] = "fifth"
df.loc["56d383b159d6e414001465e7", "answer_start"] = 336
df.loc["56d3881459d6e41400146671", "answer_text"] = "fifteenth"
df.loc["56d3881459d6e41400146671", "answer_start"] = 421
df.loc["56d3883859d6e41400146678", "question"] = "For how long was American Idol the highest rated reality show on television?"
df.loc["56d3883859d6e41400146678", "answer_text"] = "almost a decade"
df.loc["56d3883859d6e41400146678", "answer_start"] = 388
df.loc["56d3887c59d6e4140014667b", "answer_text"] = "eight"
df.loc["56d3887c59d6e4140014667b", "answer_start"] = 153
df.loc["56d38b4e59d6e414001466d7", "answer_text"] = "nine"
df.loc["56d38b4e59d6e414001466d7", "answer_start"] = 87
df.loc["56d38b4e59d6e414001466d9", "question"] = "In which year did American Idol win an award for Best Reality Competition?"
df.loc["56d38b4e59d6e414001466d9", "answer_text"] = "2011"
df.loc["56d38b4e59d6e414001466d9", "answer_start"] = 660
df.loc["56d5fc2a1c85041400946ea0", "answer_text"] = "dog breeds"
df.loc["56d5fc2a1c85041400946ea0", "answer_start"] = 285
df.loc["56d62e521c85041400946f9f", "answer_text"] = "emotional"
df.loc["56d62f3e1c85041400946fa5", "answer_text"] = "hunting"
df.loc["56dafad1e7c41114004b4bfc", "answer_start"] = 309
df.loc["56dd1cf99a695914005b94d8", "answer_text"] = "southwestern"
df.loc["56dd1dc29a695914005b94e3", "answer_start"] = 628
df.loc["56dd1ef39a695914005b94f4", "answer_start"] = 376
df.loc["56de57394396321400ee2830", "answer_text"] = "sixth"
df.loc["56deebdc3277331400b4d81f", "question"] = "Each Brigade contains how many logistics battalions?"
df.loc["56deebdc3277331400b4d81f", "answer_start"] = 466
df.loc["56de0abc4396321400ee2563", "answer_text"] = "Islamic"
df.loc["56df844f56340a1900b29cca", "answer_text"] = "700 lumens"
df.loc["56df844f56340a1900b29cca", "answer_start"] = 922
df.loc["56df865956340a1900b29ceb", "question"] = "What technique considers the amount of daylight received indoors?"
df.loc["56df865956340a1900b29ceb", "answer_text"] = "daylight factor calculation"
df.loc["56df865956340a1900b29ceb", "answer_start"] = 380
df.loc["56df95d44a1a83140091eb81", "question"] = "What is the effect of high GAI value?"
df.loc["56df95d44a1a83140091eb81", "answer_text"] = "the greater the apparent saturation or vividness of the object colors"
df.loc["56df95d44a1a83140091eb81", "answer_start"] = 315
df.loc["56dfa6de7aa994140058df9a", "question"] = "What are the medical outcomes for patients with natural scenery?"
df.loc["56dfa6de7aa994140058df9a", "answer_text"] = "shorter postoperative hospital stays , received fewer negative evaluative comments in nurses' notes , and took fewer potent analegesics"
df.loc["56dfa6de7aa994140058df9a", "answer_start"] = 248
df.loc["56dfa6de7aa994140058df9b", "question"] = "when was the study conducted by robert ulrich?"
df.loc["56dfa6de7aa994140058df9b", "answer_text"] = "1972 and 1981"
df.loc["56dfa6de7aa994140058df9b", "answer_start"] = 21
df.loc["56dfbc1d231d4119001abd48", "answer_start"] = 427
df.loc["56e462c98c00841900fbaf71", "answer_start"] = 101
df.loc["56e4744d39bdeb140034793b", "answer_start"] = 105
df.loc["56e4793839bdeb140034794f", "question"] = "How should a building fulfil the contemporary ethos?"
df.loc["56e4793839bdeb140034794f", "answer_text"] = "constructed in a manner which is environmentally friendly"
df.loc["56e4793839bdeb140034794f", "answer_start"] = 241
df.loc["56df623596943c1400a5d494", "answer_start"] = 924
df.loc["56df736f5ca0a614008f9a91", "answer_text"] = "30"
df.loc["56df736f5ca0a614008f9a91", "answer_start"] = 244
df.loc["56d4baf92ccc5a1400d8317f", "answer_text"] = "destiny fulfilled"
df.loc["56d4baf92ccc5a1400d8317f", "answer_start"] = 527
df.loc["56bfb502a10cfb140055125c", "answer_start"] = 597
df.loc["56d39a6a59d6e414001467f5", "answer_text"] = "drone basses"
df.loc["56d39a6a59d6e414001467f5", "answer_start"] = 244
df.loc["56cffba5234ae51400d9c1f1", "answer_text"] = "intuitively"
df.loc["56cffba5234ae51400d9c1f1", "answer_start"] = 443
df.loc["56cf657b4df3c31400b0d700", "answer_text"] = "two"
df.loc["56cf657b4df3c31400b0d700", "answer_start"] = 522
df.loc["56d0e42e17492d1400aab68c", "question"] = "What details of buddha's life that most scholars accept?"
df.loc["56d0e42e17492d1400aab68c", "answer_text"] = "he lived , taught and founded a monastic order"
df.loc["56d0e42e17492d1400aab68c", "answer_start"] = 400
df.loc["56d1c2d2e7d4791d0090211f", "answer_start"] = 367
df.loc["56d240dbb329da140004ec77", "answer_start"] = 118
df.loc["56de4796cffd8e1900b4b777", "answer_start"] = 83
df.loc["56de4adf4396321400ee278e", "answer_text"] = "blue dashes"
df.loc["56de8542cffd8e1900b4b9da", "answer_text"] = "western european powers"
df.loc["56de8542cffd8e1900b4b9da", "answer_start"] = 645
df.loc["56df2305c65bf219000b3f98", "answer_text"] = "general electric"
df.loc["56df2305c65bf219000b3f98", "answer_start"] = 204
df.loc["56df5cdd96943c1400a5d438", "answer_text"] = "five"
df.loc["56df5cdd96943c1400a5d438", "answer_start"] = 159
df.loc["56df5e8e96943c1400a5d44d", "question"] = "who is the largest employer in the msa area?"
df.loc["56df5e8e96943c1400a5d44d", "answer_text"] = "tinker air force base"
df.loc["56df5e8e96943c1400a5d44d", "answer_start"] = 90
df.loc["56df86855ca0a614008f9c1d", "answer_text"] = "three"
df.loc["56df86855ca0a614008f9c1d", "answer_start"] = 82
df.loc["56dfa04b38dc421700152128", "answer_start"] = 278
df.loc["56e098707aa994140058e605", "answer_start"] = 508
df.loc["56df85525ca0a614008f9bfe", "answer_text"] = "single wire"
df.loc["56df85525ca0a614008f9bfe", "answer_start"] = 198
df.loc["56df8ce34a1a83140091eb26", "answer_text"] = "one"
df.loc["56df8ce34a1a83140091eb26", "answer_start"] = 399
df.loc["56df9ad738dc4217001520c8", "answer_text"] = "nine"
df.loc["56df9ad738dc4217001520c8", "answer_start"] = 371
df.loc["56df9dbd4a1a83140091eb99", "answer_text"] = "three"
df.loc["56df9dbd4a1a83140091eb99", "answer_start"] = 192
df.loc["56dfb5977aa994140058e02d", "answer_text"] = "1907–1912"
df.loc["56dfb5977aa994140058e02d", "answer_start"] = 314
df.loc["56dfb5977aa994140058e02e", "question"] = "what are the name of the tetrahedral wings?"
df.loc["56dfb5977aa994140058e02e", "answer_text"] = "cygnet i , ii and iii"
df.loc["56dfb5977aa994140058e02e", "answer_start"] = 179
df.loc["56dfc4ba231d4119001abdce", "answer_text"] = "ten"
df.loc["56dfc4ba231d4119001abdce", "answer_start"] = 129
df.loc["56e032247aa994140058e34b", "answer_text"] = "ninth arte"
df.loc["56e06db77aa994140058e49f", "answer_text"] = "fifteen"
df.loc["56e06db77aa994140058e49f", "answer_start"] = 10
df.loc["56e079df231d4119001ac19f", "answer_text"] = "one"
df.loc["56e079df231d4119001ac19f", "answer_start"] = 66
df.loc["56e07d387aa994140058e56d", "answer_text"] = "one"
df.loc["56e07d387aa994140058e56d", "answer_start"] = 226
df.loc["56e08070231d4119001ac203", "answer_start"] = 210
df.loc["56e080dc231d4119001ac20d", "answer_text"] = "single"
df.loc["56e080dc231d4119001ac20d", "answer_start"] = 138
df.loc["56e0840c231d4119001ac23a", "answer_text"] = "three"
df.loc["56e0840c231d4119001ac23a", "answer_start"] = 77
df.loc["56e042487aa994140058e409", "answer_text"] = "⟨p⟩"
df.loc["56e042487aa994140058e409", "answer_start"] = 261
df.loc["56e06b78231d4119001ac0e7", "answer_start"] = 347
df.loc["56e0711b231d4119001ac141", "answer_start"] = 211
df.loc["56e087957aa994140058e5c1", "answer_text"] = "two"
df.loc["56e087957aa994140058e5c1", "answer_start"] = 12
df.loc["56e0914c7aa994140058e5f8", "answer_text"] = "one"
df.loc["56e0914c7aa994140058e5f8", "answer_start"] = 210
df.loc["56e0bc7b231d4119001ac364", "answer_text"] = "december 6 , 1957"
df.loc["56e0ea497aa994140058e7af", "answer_text"] = "three"
df.loc["56e0ea497aa994140058e7af", "answer_start"] = 104










Wall time: 495 ms


In [347]:
%%time
col = "qc_length"
df[col] = df["question"].str.len() + df["context"].str.len() 
df[col] = df[col].astype(np.int32)

Wall time: 86 ms


In [348]:
%%time
col = "a_length"
df[col] = df["answer_text"].str.len()
df[col] = df[col].astype(np.int32)

Wall time: 48 ms


In [349]:
def word_length(cols: Iterable) -> Callable:
    def f(row) -> int:
        res = 0
        for col in cols:
            res += len(row[col].split())
        return res
    
    return f

In [350]:
%%time
col = "qc_word_length"
df[col] = df.progress_apply(word_length(["question", "context"]), axis=1)
df[col] = df[col].astype(np.int32)

100%|████████████████████████| 130319/130319 [00:02<00:00, 49240.33it/s]

Wall time: 2.65 s


In [351]:
%%time
col = "a_word_length"
df[col] = df.progress_apply(word_length(["answer_text"]), axis=1)
df[col] = df[col].astype(np.int32)

100%|███████████████████████| 130319/130319 [00:01<00:00, 108418.61it/s]

Wall time: 1.21 s


In [352]:
#pretrained_dir = "../pretrained/google/electra-small-discriminator"
#tokenizer = AutoTokenizer.from_pretrained(pretrained_dir, model_max_length=512)
#print(f"{repr(tokenizer)}\n{tokenizer.model_input_names}")

In [353]:
#pretrained_dir = "../pretrained/albert-base-v2"
#sp_tokenizer = AutoTokenizer.from_pretrained(pretrained_dir, model_max_length=512)
#print(f"{repr(sp_tokenizer)}\n{sp_tokenizer.model_input_names}")

In [354]:
#pretrained_dir = "../pretrained/distilroberta-base"
#bpe_tokenizer = AutoTokenizer.from_pretrained(pretrained_dir, model_max_length=512)
#print(f"{repr(bpe_tokenizer)}\n{bpe_tokenizer.model_input_names}")

In [355]:
#%%time
#x = tokenizer(questions, contexts)
#print(f"{repr(x.keys())}\nlen={len(x['input_ids'])}")
#col = "qc_wp_length"
#df[col] = [len(v) for v in x["input_ids"]]
#df[col] = df[col].astype(np.int16)

In [356]:
#%%time
#x = sp_tokenizer(questions, contexts)
#print(f"{repr(x.keys())}\nlen={len(x['input_ids'])}")
#col = "qc_sp_length"
#df[col] = [len(v) for v in x["input_ids"]]
#df[col] = df[col].astype(np.int16)

In [357]:
#%%time
#x = bpe_tokenizer(questions, contexts)
#print(f"{repr(x.keys())}\nlen={len(x['input_ids'])}")
#col = "qc_bpe_length"
#df[col] = [len(v) for v in x["input_ids"]]
#df[col] = df[col].astype(np.int16)

In [358]:
#cols = ["qc_length", "a_length", "qc_word_length", "a_word_length", 
#        "qc_wp_length", "qc_sp_length", "qc_bpe_length"]
#df[cols].describe(percentiles=percentiles)

In [359]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 130319 entries, 56be85543aeaaa14008c9063 to 5a7e070b70df9f001a87543d
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   id              130319 non-null  object
 1   title           130319 non-null  object
 2   question        130319 non-null  object
 3   answer_text     130319 non-null  object
 4   answer_start    130319 non-null  int16 
 5   context         130319 non-null  object
 6   qc_length       130319 non-null  int32 
 7   a_length        130319 non-null  int32 
 8   qc_word_length  130319 non-null  int32 
 9   a_word_length   130319 non-null  int32 
dtypes: int16(1), int32(4), object(5)
memory usage: 12.2+ MB


In [360]:
%%time
df.to_parquet("output/train.parquet", index=False)

Wall time: 295 ms
